 ###  Convolutional Neural Networks for Emotion Detection from Facial Expressions

.

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define constants
IMG_SIZE = 48  # resize images to 48x48 pixels
CATEGORIES = ["Angry", "Happy", "Sad"]  # emotion categories

# Load and preprocess images
data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(".", category)  # path to the category directory
    class_num = CATEGORIES.index(category)  # assign a number to each category
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)  # read image
            resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image
            data.append(resized_array)
            labels.append(class_num)
        except Exception as e:
            pass  # if any images fail to load, skip them

# Convert lists to numpy arrays and normalize pixel values
data = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
labels = np.array(labels)

# Split data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)


#### Define the model architecture

.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(CATEGORIES), activation='softmax'))


#### Compile the model

.

In [3]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#### Train the model

.

In [4]:
model.fit(trainX, trainY, batch_size=32, epochs=10, validation_data=(testX, testY))

Epoch 1/10
7/7 [==============================] - 5s 179ms/step - loss: 1.1719 - accuracy: 0.3507 - val_loss: 1.0906 - val_accuracy: 0.3774
Epoch 2/10
7/7 [==============================] - 1s 95ms/step - loss: 1.0863 - accuracy: 0.3649 - val_loss: 1.0942 - val_accuracy: 0.3208
Epoch 3/10
7/7 [==============================] - 1s 94ms/step - loss: 1.0850 - accuracy: 0.3649 - val_loss: 1.0881 - val_accuracy: 0.3208
Epoch 4/10
7/7 [==============================] - 1s 94ms/step - loss: 1.0786 - accuracy: 0.3697 - val_loss: 1.0785 - val_accuracy: 0.4717
Epoch 5/10
7/7 [==============================] - 1s 99ms/step - loss: 1.0463 - accuracy: 0.5166 - val_loss: 1.0700 - val_accuracy: 0.4151
Epoch 6/10
7/7 [==============================] - 1s 93ms/step - loss: 1.0269 - accuracy: 0.5355 - val_loss: 1.0467 - val_accuracy: 0.3774
Epoch 7/10
7/7 [==============================] - 1s 107ms/step - loss: 0.9911 - accuracy: 0.4882 - val_loss: 1.0186 - val_accuracy: 0.4906
Epoch 8/10
7/7 [=========

#### Save the model

.

In [ ]:
model.save('emotion_model.h5')

#### Reload the model

.

In [5]:
from tensorflow.keras.models import load_model

loaded_model = load_model('emotion_model.h5')


#### Evaluate the model

.

In [6]:
from sklearn.metrics import classification_report

# Generate predictions on the test set
y_pred = loaded_model.predict(testX)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate a classification report
print(classification_report(testY, y_pred_classes, target_names=CATEGORIES))

2/2 [==============================] - 0s 21ms/step
              precision    recall  f1-score   support

       Angry       0.45      0.29      0.36        17
       Happy       0.46      0.85      0.60        20
         Sad       0.60      0.19      0.29        16

    accuracy                           0.47        53
   macro avg       0.50      0.44      0.41        53
weighted avg       0.50      0.47      0.43        53



#### See the result with new images

.

In [7]:

# Load the trained model
model = load_model('emotion_model.h5')

# Load a new image
img_path = 'new image.jpg'  # replace with your image path
img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # read image
resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image

# Preprocess the image
input_data = np.array(resized_array).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0

# Make a prediction
prediction = model.predict(input_data)
predicted_class = CATEGORIES[np.argmax(prediction)]

print(f'The model predicts that the emotion in the image is: {predicted_class}')

1/1 [==============================] - 0s 171ms/step
The model predicts that the emotion in the image is: Angry


.

#### Now try this model with your new or unseen images and see the result. 

.

### Thank You